In [1]:
#! pip install pygeocoder
#from pygeocoder import Geocoder - i tried these two; simple 2 line of code, gave me error of 
#GeocoderError: Error REQUEST_DENIED
#Query: https://maps.google.com/maps/api/geocode/json?latlng=50.122064%2C5.532898&sensor=false&bounds=&region=&language=
#also requires setting up API keys, tokens etc
#they probably charge for that many records

#import geopandas as gpd
#import geopy
#from geopy.geocoders import Nominatim
#from geopy.extra.rate_limiter import RateLimiter
#using  Openstreetmap, an open source map which coinmap itself is based on but i get following error message
#Solving environment: failed with initial frozen solve. Retrying with flexible solve.

# My third attempt with googlemaps and getting information using google API key worked.
#! pip install googlemaps
import requests
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
import json
import googlemaps
import time

In [2]:
df = requests.get("https://coinmap.org/api/v1/venues").json()  # importing json file from coinmap.org
df = pd.json_normalize(df, record_path = ['venues']) # normalize rearranges and puts json dictionary file in order by grabbing 'venues' key
df.shape  # 22,278 Coinmap venues from Dec. 26, 2012 to Jun. 2, 2021 which equals the of records in our df

(22278, 8)

In [3]:
df_usa = df[df['lon'] <= -66.949471] # eliminates world countries anything to the left of eastern most point of Maine
df_usa = df_usa[df_usa['lat'] >= 18.464825] # eliminates countries south of the southern tip of Hawaii, Florida Keys is a little above this.

In [4]:
df_usa.shape # Now the file reduced in size, still includes Canada and Alaska, some parts of Mexico & Puerto Rico

(6912, 8)

In [5]:
df_usa = df_usa.reset_index(drop=True)

In [6]:
df_usa.dtypes

id                       int64
lat                    float64
lon                    float64
category                object
name                    object
created_on               int64
geolocation_degrees     object
promoted                object
dtype: object

In [7]:
df_usa.head(50) # data from Coinmap.org

,id,lat,lon,category,name,created_on,geolocation_degrees,promoted
0,15,48.437747,-123.312411,grocery,Slaters Meats,1394219130,"123°18'44.68""W, 48°26'15.89""N",NaN
1,8,39.261575,-121.016567,default,SierraHosts.net,1386794770,"121°0'59.64""W, 39°15'41.67""N",NaN
2,10,39.714651,-104.936268,default,Costa Rica Travel Market LLC,1389447527,"104°56'10.57""W, 39°42'52.74""N",NaN
3,12,42.362679,-71.086398,shopping,MIT Coop,1409879886,"71°5'11.03""W, 42°21'45.65""N",NaN
4,6,37.770518,-122.450413,default,Deborah Hodgson Vamp Couture,1385666967,"122°27'1.49""W, 37°46'13.87""N",NaN
5,8226,40.752564,-86.346627,shopping,Stack-Tech Computer Repair,1465956215,"86°20'47.86""W, 40°45'9.23""N",NaN
6,3928,37.958075,-122.499287,cafe,Nova Rosti,1399646864,"122°29'57.43""W, 37°57'29.07""N",NaN
7,50,39.986389,-83.005556,default,Columbus Worship Center,1386095442,"83°0'20.0""W, 39°59'11.0""N",NaN
8,51,41.404722,-74.325000,default,Saint John The Evangelist Roman Catholic Church,1383436754,"74°19'30.0""W, 41°24'17.0""N",NaN
9,52,30.409483,-91.068455,default,Dezins Interactive,1385449366,"91°4'6.44""W, 30°24'34.14""N",NaN


In [1]:
keyer = input("What is your API Key ? : ") # API key in here, dont store here
gmaps = googlemaps.Client(key = keyer)

stn = ['street_number'] # street number                      
rt = ['route'] #----------> # route
lp = ['locality', 'political'] # city
aalp2 = ['administrative_area_level_2', 'political'] # county
aalp1 = ['administrative_area_level_1', 'political'] # state
cntry = ['country', 'political'] # country
pc = ['postal_code']  # zip Code

for row in range(len(df_usa)):
    reverse_geocode_result = gmaps.reverse_geocode((df_usa.at[row, "lat"], df_usa.at[row, "lon"]))
                                                                    # Looks up an address with reverse geocoding
        # time.sleep(2) # sleep 2 seconds due to reverse geocoding request having broadband connection laterncy from Google
    
    try:  # try and its exception part employed to capture error conditions from code while accessing info from Google servers
        if reverse_geocode_result[0]['address_components'][0]['types'] == stn:
            df_usa.at[row, "Street_Num"] = reverse_geocode_result[0]['address_components'][0]['long_name']
        elif reverse_geocode_result[0]['address_components'][0]['types'] == rt:
            df_usa.at[row, "Street_Name"] = reverse_geocode_result[0]['address_components'][0]['long_name']
        elif reverse_geocode_result[0]['address_components'][0]['types'] == lp:
            df_usa.at[row, "City"] = reverse_geocode_result[0]['address_components'][0]['long_name']
        elif reverse_geocode_result[0]['address_components'][0]['types'] == aalp2:
            df_usa.at[row, "County"] = reverse_geocode_result[0]['address_components'][0]['long_name']
        elif reverse_geocode_result[0]['address_components'][0]['types'] == aalp1:
            df_usa.at[row, "State"] = reverse_geocode_result[0]['address_components'][0]['long_name']
        elif reverse_geocode_result[0]['address_components'][0]['types'] == cntry:
            df_usa.at[row, "Country"] = reverse_geocode_result[0]['address_components'][0]['long_name']
        elif reverse_geocode_result[0]['address_components'][0]['types'] == pc:
            df_usa.at[row, "Zip"] = reverse_geocode_result[0]['address_components'][0]['long_name']
        if reverse_geocode_result[0]['address_components'][1]['types'] == stn:
            df_usa.at[row, "Street_Num"] = reverse_geocode_result[0]['address_components'][1]['long_name']
        elif reverse_geocode_result[0]['address_components'][1]['types'] == rt:
            df_usa.at[row, "Street_Name"] = reverse_geocode_result[0]['address_components'][1]['long_name']
        elif reverse_geocode_result[0]['address_components'][1]['types'] == lp:
            df_usa.at[row, "City"] = reverse_geocode_result[0]['address_components'][1]['long_name']
        elif reverse_geocode_result[0]['address_components'][1]['types'] == aalp2:
            df_usa.at[row, "County"] = reverse_geocode_result[0]['address_components'][1]['long_name']
        elif reverse_geocode_result[0]['address_components'][1]['types'] == aalp1:
            df_usa.at[row, "State"] = reverse_geocode_result[0]['address_components'][1]['long_name']
        elif reverse_geocode_result[0]['address_components'][1]['types'] == cntry:
            df_usa.at[row, "Country"] = reverse_geocode_result[0]['address_components'][1]['long_name']
        elif reverse_geocode_result[0]['address_components'][1]['types'] == pc:
            df_usa.at[row, "Zip"] = reverse_geocode_result[0]['address_components'][1]['long_name']
        if reverse_geocode_result[0]['address_components'][2]['types'] == stn:
            df_usa.at[row, "Street_Num"] = reverse_geocode_result[0]['address_components'][2]['long_name']
        elif reverse_geocode_result[0]['address_components'][2]['types'] == rt:
            df_usa.at[row, "Street_Name"] = reverse_geocode_result[0]['address_components'][2]['long_name']
        elif reverse_geocode_result[0]['address_components'][2]['types'] == lp:
            df_usa.at[row, "City"] = reverse_geocode_result[0]['address_components'][2]['long_name']
        elif reverse_geocode_result[0]['address_components'][2]['types'] == aalp2:
            df_usa.at[row, "County"] = reverse_geocode_result[0]['address_components'][2]['long_name']
        elif reverse_geocode_result[0]['address_components'][2]['types'] == aalp1:
            df_usa.at[row, "State"] = reverse_geocode_result[0]['address_components'][2]['long_name']
        elif reverse_geocode_result[0]['address_components'][2]['types'] == cntry:
            df_usa.at[row, "Country"] = reverse_geocode_result[0]['address_components'][2]['long_name']
        elif reverse_geocode_result[0]['address_components'][2]['types'] == pc:
            df_usa.at[row, "Zip"] = reverse_geocode_result[0]['address_components'][2]['long_name']
        if reverse_geocode_result[0]['address_components'][3]['types'] == stn:
            df_usa.at[row, "Street_Num"] = reverse_geocode_result[0]['address_components'][3]['long_name']
        elif reverse_geocode_result[0]['address_components'][3]['types'] == rt:
            df_usa.at[row, "Street_Name"] = reverse_geocode_result[0]['address_components'][3]['long_name']
        elif reverse_geocode_result[0]['address_components'][3]['types'] == lp:
            df_usa.at[row, "City"] = reverse_geocode_result[0]['address_components'][3]['long_name']
        elif reverse_geocode_result[0]['address_components'][3]['types'] == aalp2:
            df_usa.at[row, "County"] = reverse_geocode_result[0]['address_components'][3]['long_name']
        elif reverse_geocode_result[0]['address_components'][3]['types'] == aalp1:
            df_usa.at[row, "State"] = reverse_geocode_result[0]['address_components'][3]['long_name']
        elif reverse_geocode_result[0]['address_components'][3]['types'] == cntry:
            df_usa.at[row, "Country"] = reverse_geocode_result[0]['address_components'][3]['long_name']
        elif reverse_geocode_result[0]['address_components'][3]['types'] == pc:
            df_usa.at[row, "Zip"] = reverse_geocode_result[0]['address_components'][3]['long_name']
        if reverse_geocode_result[0]['address_components'][4]['types'] == stn:
            df_usa.at[row, "Street_Num"] = reverse_geocode_result[0]['address_components'][4]['long_name']
        elif reverse_geocode_result[0]['address_components'][4]['types'] == rt:
            df_usa.at[row, "Street_Name"] = reverse_geocode_result[0]['address_components'][4]['long_name']
        elif reverse_geocode_result[0]['address_components'][4]['types'] == lp:
            df_usa.at[row, "City"] = reverse_geocode_result[0]['address_components'][4]['long_name']
        elif reverse_geocode_result[0]['address_components'][4]['types'] == aalp2:
            df_usa.at[row, "County"] = reverse_geocode_result[0]['address_components'][4]['long_name']
        elif reverse_geocode_result[0]['address_components'][4]['types'] == aalp1:
            df_usa.at[row, "State"] = reverse_geocode_result[0]['address_components'][4]['long_name']
        elif reverse_geocode_result[0]['address_components'][4]['types'] == cntry:
            df_usa.at[row, "Country"] = reverse_geocode_result[0]['address_components'][4]['long_name']
        elif reverse_geocode_result[0]['address_components'][4]['types'] == pc:
            df_usa.at[row, "Zip"] = reverse_geocode_result[0]['address_components'][4]['long_name']
        if reverse_geocode_result[0]['address_components'][5]['types'] == stn:
            df_usa.at[row, "Street_Num"] = reverse_geocode_result[0]['address_components'][5]['long_name']
        elif reverse_geocode_result[0]['address_components'][5]['types'] == rt:
            df_usa.at[row, "Street_Name"] = reverse_geocode_result[0]['address_components'][5]['long_name']
        elif reverse_geocode_result[0]['address_components'][5]['types'] == lp:
            df_usa.at[row, "City"] = reverse_geocode_result[0]['address_components'][5]['long_name']
        elif reverse_geocode_result[0]['address_components'][5]['types'] == aalp2:
            df_usa.at[row, "County"] = reverse_geocode_result[0]['address_components'][5]['long_name']
        elif reverse_geocode_result[0]['address_components'][5]['types'] == aalp1:
            df_usa.at[row, "State"] = reverse_geocode_result[0]['address_components'][5]['long_name']
        elif reverse_geocode_result[0]['address_components'][5]['types'] == cntry:
            df_usa.at[row, "Country"] = reverse_geocode_result[0]['address_components'][5]['long_name']
        elif reverse_geocode_result[0]['address_components'][5]['types'] == pc:
            df_usa.at[row, "Zip"] = reverse_geocode_result[0]['address_components'][5]['long_name']
        if reverse_geocode_result[0]['address_components'][6]['types'] == stn:
            df_usa.at[row, "Street_Num"] = reverse_geocode_result[0]['address_components'][6]['long_name']
        elif reverse_geocode_result[0]['address_components'][6]['types'] == rt:
            df_usa.at[row, "Street_Name"] = reverse_geocode_result[0]['address_components'][6]['long_name']
        elif reverse_geocode_result[0]['address_components'][6]['types'] == lp:
            df_usa.at[row, "City"] = reverse_geocode_result[0]['address_components'][6]['long_name']
        elif reverse_geocode_result[0]['address_components'][6]['types'] == aalp2:
            df_usa.at[row, "County"] = reverse_geocode_result[0]['address_components'][6]['long_name']
        elif reverse_geocode_result[0]['address_components'][6]['types'] == aalp1:
            df_usa.at[row, "State"] = reverse_geocode_result[0]['address_components'][6]['long_name']
        elif reverse_geocode_result[0]['address_components'][6]['types'] == cntry:
            df_usa.at[row, "Country"] = reverse_geocode_result[0]['address_components'][6]['long_name']
        elif reverse_geocode_result[0]['address_components'][6]['types'] == pc:
            df_usa.at[row, "Zip"] = reverse_geocode_result[0]['address_components'][6]['long_name']
        if reverse_geocode_result[0]['address_components'][7]['types'] == stn:
            df_usa.at[row, "Street_Num"] = reverse_geocode_result[0]['address_components'][7]['long_name']
        elif reverse_geocode_result[0]['address_components'][7]['types'] == rt:
            df_usa.at[row, "Street_Name"] = reverse_geocode_result[0]['address_components'][7]['long_name']
        elif reverse_geocode_result[0]['address_components'][7]['types'] == lp:
            df_usa.at[row, "City"] = reverse_geocode_result[0]['address_components'][7]['long_name']
        elif reverse_geocode_result[0]['address_components'][7]['types'] == aalp2:
            df_usa.at[row, "County"] = reverse_geocode_result[0]['address_components'][7]['long_name']
        elif reverse_geocode_result[0]['address_components'][7]['types'] == aalp1:
            df_usa.at[row, "State"] = reverse_geocode_result[0]['address_components'][7]['long_name']
        elif reverse_geocode_result[0]['address_components'][7]['types'] == cntry:
            df_usa.at[row, "Country"] = reverse_geocode_result[0]['address_components'][7]['long_name']
        elif reverse_geocode_result[0]['address_components'][7]['types'] == pc:
            df_usa.at[row, "Zip"] = reverse_geocode_result[0]['address_components'][7]['long_name']
        if reverse_geocode_result[0]['address_components'][8]['types'] == stn:
            df_usa.at[row, "Street_Num"] = reverse_geocode_result[0]['address_components'][8]['long_name']
        elif reverse_geocode_result[0]['address_components'][8]['types'] == rt:
            df_usa.at[row, "Street_Name"] = reverse_geocode_result[0]['address_components'][8]['long_name']
        elif reverse_geocode_result[0]['address_components'][8]['types'] == lp:
            df_usa.at[row, "City"] = reverse_geocode_result[0]['address_components'][8]['long_name']
        elif reverse_geocode_result[0]['address_components'][8]['types'] == aalp2:
            df_usa.at[row, "County"] = reverse_geocode_result[0]['address_components'][8]['long_name']
        elif reverse_geocode_result[0]['address_components'][8]['types'] == aalp1:
            df_usa.at[row, "State"] = reverse_geocode_result[0]['address_components'][8]['long_name']
        elif reverse_geocode_result[0]['address_components'][8]['types'] == cntry:
            df_usa.at[row, "Country"] = reverse_geocode_result[0]['address_components'][8]['long_name']
        elif reverse_geocode_result[0]['address_components'][8]['types'] == pc:
            df_usa.at[row, "Zip"] = reverse_geocode_result[0]['address_components'][8]['long_name']
        if reverse_geocode_result[0]['address_components'][9]['types'] == stn:
            df_usa.at[row, "Street_Num"] = reverse_geocode_result[0]['address_components'][9]['long_name']
        elif reverse_geocode_result[0]['address_components'][9]['types'] == rt:
            df_usa.at[row, "Street_Name"] = reverse_geocode_result[0]['address_components'][9]['long_name']
        elif reverse_geocode_result[0]['address_components'][9]['types'] == lp:
            df_usa.at[row, "City"] = reverse_geocode_result[0]['address_components'][9]['long_name']
        elif reverse_geocode_result[0]['address_components'][9]['types'] == aalp2:
            df_usa.at[row, "County"] = reverse_geocode_result[0]['address_components'][9]['long_name']
        elif reverse_geocode_result[0]['address_components'][9]['types'] == aalp1:
            df_usa.at[row, "State"] = reverse_geocode_result[0]['address_components'][9]['long_name']
        elif reverse_geocode_result[0]['address_components'][9]['types'] == cntry:
            df_usa.at[row, "Country"] = reverse_geocode_result[0]['address_components'][9]['long_name']
        elif reverse_geocode_result[0]['address_components'][9]['types'] == pc:
            df_usa.at[row, "Zip"] = reverse_geocode_result[0]['address_components'][9]['long_name']
    except:
        continue

df_usa.to_csv("C:/Users/TaxCollector/Desktop/Data Science/DS Program Course/Project Data Sets and Documents/CoinMap.csv", index=False)

What is your API Key ? : xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


NameError: name 'googlemaps' is not defined

In [10]:
df_usa.tail(50)

,id,lat,lon,category,name,created_on,geolocation_degrees,promoted,Street_Num,Street_Name,City,County,State,Country,Zip
6862,40929,34.142644,-84.010257,atm,Shell,1614241632,"84°0'36.92""W, 34°8'33.52""N",True,2380,Buford Dam Road,Buford,Gwinnett County,Georgia,United States,30518
6863,40930,38.113485,-85.690717,atm,Silver Heights Coin Laundry,1614241635,"85°41'26.58""W, 38°6'48.55""N",True,9320,Blue Lick Road,Louisville,Jefferson County,Kentucky,United States,40229
6864,58070,29.029078,-99.050574,atm,Road Ranger,1615286128,"99°3'2.07""W, 29°1'44.68""N",True,NaN,Interstate 35,Moore,Frio County,Texas,United States,78057
6865,58071,28.046792,-99.349280,atm,Road Ranger,1615286128,"99°20'57.41""W, 28°2'48.45""N",True,2009,Interstate 35 Frontage Road,Encinal,La Salle County,Texas,United States,78019
6866,58072,32.356495,-86.213646,atm,Royal Package Store,1615286135,"86°12'49.13""W, 32°21'23.38""N",True,5761,Woodmere Boulevard,Montgomery,Montgomery County,Alabama,United States,36117
6867,58073,33.235663,-87.481425,atm,S&M Food Mart - Tuscaloosa,1615286139,"87°28'53.13""W, 33°14'8.39""N",True,1800,Crescent Ridge Road Northeast,Tuscaloosa,Tuscaloosa County,Alabama,United States,35404
6868,58074,41.246906,-81.344929,atm,Sammy's Food Mart,1615286141,"81°20'41.74""W, 41°14'48.86""N",True,9418,Ohio 43,Streetsboro,Portage County,Ohio,United States,44241
6869,46629,38.189411,-84.282901,atm,Shell,1614616012,"84°16'58.44""W, 38°11'21.88""N",True,4107,Lexington Road,Paris,Bourbon County,Kentucky,United States,40361
6870,46630,38.001839,-84.545831,atm,Shell,1614616013,"84°32'44.99""W, 38°0'6.62""N",True,3197,Clays Mill Road,Lexington,Fayette County,Kentucky,United States,40503
6871,46631,37.736575,-84.326878,atm,Shell,1614616013,"84°19'36.76""W, 37°44'11.67""N",True,1074,Barnes Mill Road,Richmond,Madison County,Kentucky,United States,40475
